### Annotation instructions:

1. Rotate the image to find a position, where the filament is clearly visible
2. Draw a line over the filament, by holding "Control" and clicking with the mouse: this will draw a polygon with potential filament locations
3. Rotate the image to view the filament from another angle and repeat step 2
4. Rotate the image again: this will calculate the filament position from the intersection of the two polygons
5. Repeat steps 1-4 for other filaments

To delete the last added shape (polygon or filament), press "d"

In [ ]:
import napari
import numpy as np
from skimage import io
from scipy import ndimage

from filament_annotation_3d.annotator import *
from filament_annotation_3d.utils.sim import create_random_lines

In [ ]:
output_fn = 'annotations.csv'

In [ ]:
# generate a randon line image and display with napari
viewer = napari.view_image(create_random_lines(100, 10), ndisplay=3) 


# add a layer for annotations
an_layer = add_annotation_layer(viewer) 

# activate annotation of filaments
annotate_filaments(an_layer, output_fn, point_size=1,
                   image_layer=viewer.layers[0], sigma=2., 
                   neighborhood_radius=5, decay_sigma=2.) 